In [1]:
%pylab inline

/Users/piyushghai/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [3]:
#All the imports in one place

import time
import datetime

import cPickle as pickle
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pylab
import re
import scipy as sp
import seaborn

from gensim import corpora, models
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import *
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, roc_auc_score
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter
import os
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")


plt.rc('figure', figsize=(10,6))
seaborn.set()
colors = seaborn.color_palette()




In [5]:
all_text_train = pickle.load (open("all_text_train.p", "rb"))

all_text_test = pickle.load (open("all_text_test.p", "rb"))

topic_dist_train_all_stars = pickle.load(open("topic_dist_train_all_stars.p","rb"))

topic_dist_test_all_stars = pickle.load(open("topic_dist_test_all_stars.p","rb"))


In [6]:
## Defining Sentiment : If review is < 3 stars, label it as -ve sent, else +ve sent
def getSentiment(x):
    if x < 3.0:
        return 0
    else:
        return 1

In [11]:
tfidfvectorizer = TfidfVectorizer()

In [7]:
topic_dist_train_all_stars['Sentiment'] = topic_dist_train_all_stars['Star'].map(getSentiment)
topic_dist_test_all_stars['Sentiment'] = topic_dist_test_all_stars['Star'].map(getSentiment)

In [12]:
sentimentTextTrain = tfidfvectorizer.fit_transform(all_text_train)
sentimentTextTest = tfidfvectorizer.transform(all_text_test)

sentimentLabelTrain = topic_dist_train_all_stars['Sentiment']
sentimentLabelTest = topic_dist_test_all_stars['Sentiment']

In [ ]:
classifier = LogisticRegression().fit(sentimentTextTrain, sentimentLabelTrain)

ySentimentTrain = classifier.predict(sentimentTextTrain)
ySentimentTest = classifier.predict(sentimentTextTest)

topic_dist_train_all_stars['Sentiment_Predicted'] = ySentimentTrain
topic_dist_test_all_stars['Sentiment_Predicted'] = ySentimentTest

In [ ]:
features = list(topic_dist_train_all_stars.columns[:totalTopics])
features.append(topic_dist_train_all_stars.columns[totalTopics+2])

In [ ]:
x_train = topic_dist_train_all_stars[features]
y_train = topic_dist_train_all_stars['Star']

x_test = topic_dist_test_all_stars[features]
y_test = topic_dist_test_all_stars['Star'] 

In [ ]:
classifiers = [MultinomialNB(), LogisticRegression(), RandomForestClassifier(n_estimators=100, n_jobs=2), AdaBoostClassifier(n_estimators=100)]
classifiers_names = ['Multinomial Naive Bayes', 'Logistic Regression', 'Random Forest', 'AdaBoost']

LdaSentimentResults = {}
for (i, clf_) in enumerate(classifiers):
    clf = clf_.fit(x_train, y_train)
    preds = clf.predict(x_test)
    
    precision = metrics.precision_score(y_test, preds)
    recall = metrics.recall_score(y_test, preds)
    f1 = metrics.f1_score(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    report = classification_report(y_test, preds)
    matrix = metrics.confusion_matrix(y_test, preds, labels=starsGroup.groups.keys())
    
    data = {'precision':precision,
            'recall':recall,
            'f1_score':f1,
            'accuracy':accuracy,
            'clf_report':report,
            'clf_matrix':matrix,
            'y_predicted':preds}
    
    LdaSentimentResults[classifiers_names[i]] = data

In [14]:
sys.stdout = open("lda_sentiment_results.txt", 'a')

In [ ]:
cols = ['precision', 'recall', 'f1_score', 'accuracy']
pd.DataFrame(LdaSentimentResults).T[cols].T

for model, val in LdaSentimentResults.iteritems():
    print '-------'+'-'*len(model)
    print 'MODEL:', model
    print '-------'+'-'*len(model)
    print 'The precision for this classifier is ' + str(val['precision'])
    print 'The recall for this classifier is    ' + str(val['recall'])
    print 'The f1 for this classifier is        ' + str(val['f1_score'])
    print 'The accuracy for this classifier is  ' + str(val['accuracy'])
    print 'The confusion matrix for this classifier is  \n' + str(val['clf_matrix'])
    print '\nHere is the classification report:'
    print val['clf_report']